In [4]:
# 平替向量化写法
import pandas as pd
import requests
import json
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored
from zhipuai import ZhipuAI
from openai import OpenAI 
import os
api_key = os.environ.get('ZHIPU_KEY')

glm_4_url = "https://open.bigmodel.cn/api/paas/v4/chat/completions"
model = 'glm-4'
#################### zhipu的第三方兼容写法
# client = OpenAI(
#     api_key=api_key,
#     base_url="https://open.bigmodel.cn/api/paas/v4/"
# ) 


## 1.定义工具函数

In [5]:
def get_current_weather(location):
    res = requests.get(f'https://v1.yiketianqi.com/free/month?unescape=1&appid=58426383&appsecret=dOMQ3zQ4&city={location}')
    res = eval(res.text)
    if res.get('errmsg'):
        return res.get('errmsg')
    return res['data'][0]

def get_next_x_days_weather(location, x):
    x = int(x)
    res = requests.get(f'https://v1.yiketianqi.com/free/month?unescape=1&appid=58426383&appsecret=dOMQ3zQ4&city={location}')
    res = eval(res.text)
    if res.get('errmsg'):
        return res.get('errmsg')
    return res['data'][1:x+1]

tools = [
    {
        'type':'function',
        'function': {
            'name': 'get_current_weather',
            'description': '根据地名，查询当天的天气情况',
            'parameters': {
                'type': 'object',
                'properties':{
                    'location': {
                        'type': 'string',
                        'description': '地名,请剔除"市"或"区"等后缀'
                    }
                },
                'required': ['location']
            }
        }
    },
    {
        'type':'function',
        'function': {
            'name': 'get_next_x_days_weather',
            'description': '根据地名，查询未来x天的天气情况',
            'parameters': {
                'type': 'object',
                'properties': {
                    'location': {
                        'type': 'string',
                        'description': '地名,请剔除"市"或"区"等后缀'
                    },
                    'x': {
                        'type': 'integer',
                        'description': '未来天数'
                    }
                },
                'required': ['location', 'x']
            }
        }
    }
]

## 2.定义工具函数及打印函数

In [6]:
# 定义一个函数pretty_print_conversation，用于打印消息对话内容
def pretty_print_conversation(messages):

    # 为不同角色设置不同的颜色
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "tool": "magenta",
    }

    # 遍历消息列表
    for message in messages:

        # 如果消息的角色是"system"，则用红色打印“content”
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))

        # 如果消息的角色是"user"，则用绿色打印“content”
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))

        # 如果消息的角色是"assistant"，并且消息中包含"function_call"，则用蓝色打印"function_call"
        elif message["role"] == "assistant" and message.get("tool_calls"):
            print(colored(f"assistant[tool_calls]: {message['tool_calls']}\n", role_to_color[message["role"]]))

        # 如果消息的角色是"assistant"，但是消息中不包含"function_call"，则用蓝色打印“content”
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant[content]: {message['content']}\n", role_to_color[message["role"]]))

        # 如果消息的角色是"function"，则用品红色打印“function”
        elif message["role"] == "tool":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))

# 使用了retry库，指定在请求失败时的重试策略。
# 这里设定的是指数等待（wait_random_exponential），时间间隔的最大值为40秒，并且最多重试3次（stop_after_attempt(3)）。
# 定义一个函数chat_completion_request，主要用于发送 聊天补全 请求到OpenAI服务器
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, tools=None, tool_choice=None, model=model):

    # 设定请求的header信息，包括 API_KEY
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}",
    }

    # 设定请求的JSON数据，包括GPT 模型名和要进行补全的消息
    json_data = {"model": model, "messages": messages}

    # 如果传入了functions，将其加入到json_data中
    if tools is not None:
        json_data.update({"tools": tools})

    # 如果传入了function_call，将其加入到json_data中
    if tool_choice is not None:
        json_data.update({"tool_choice": tool_choice})

    # 尝试发送POST请求到OpenAI服务器的chat/completions接口
    try:
        response = requests.post(
            glm_4_url,
            headers=headers,
            json=json_data,
        )
        # 返回服务器的响应
        return response

    # 如果发送请求或处理响应时出现异常，打印异常信息并返回
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

## 3.定义执行函数

In [7]:
def excute_tool(response_message):
    # 处理函数调用结果，根据模型返回参数，调用对应的函数。
    # 调用函数返回结果后构造tool message，再次调用模型，将函数结果输入模型
    # 模型会将函数调用结果以自然语言格式返回给用户。
    tool_call = response_message['tool_calls'][0]
    args = tool_call['function']['arguments']
    function_result = {}
    if tool_call['function']['name'] == "get_current_weather":
        function_result = get_current_weather(**json.loads(args))
    if tool_call['function']['name'] == "get_next_x_days_weather":
        function_result = get_next_x_days_weather(**json.loads(args))
    return function_result

In [ ]:
常规回复结构
{'choices': [{'finish_reason': 'stop',
   'index': 0,
   'message': {'content': '您需要查询广州当前的天气情况，我可以帮助您完成这个请求。但在此之前，我需要确认您是否需要知道详细的天气信息，比如温度、湿度、风力等。如果您需要这些详细信息，请告知我，我将为您提供最新的广州天气状况。',
    'role': 'assistant'}}],
 'created': 1724663004,
 'id': '2024082617032246181c513e1d4f37',
 'model': 'glm-4',
 'request_id': '2024082617032246181c513e1d4f37',
 'usage': {'completion_tokens': 55, 'prompt_tokens': 34, 'total_tokens': 89}}

function_call回复结构
{'choices': [{'finish_reason': 'tool_calls',
   'index': 0,
   'message': {'content': '',
    'role': 'assistant',
    'tool_calls': [{'function': {'arguments': '{"location":"广州"}',
       'name': 'get_current_weather'},
      'id': 'call_20240826170448dbea3e22b68f4c2a',
      'index': 0,
      'type': 'function'}]}}],
 'created': 1724663090,
 'id': '20240826170448dbea3e22b68f4c2a',
 'model': 'glm-4',
 'request_id': '20240826170448dbea3e22b68f4c2a',
 'usage': {'completion_tokens': 15, 'prompt_tokens': 283, 'total_tokens': 298}}

## 4.构造案例

In [8]:
messages = [{'role': 'system', 'content': '你是一个天气预报大师，你需要先检查用户的描述中是否有提供所在地'}]
messages.append({'role': 'user', 'content': '今天天气怎么样?'})

# 使用定义的chat_completion_request函数发起一个请求，传入messages和functions作为参数
# tool_chioce = {"type": "function", "function": {"name": "get_next_x_days_weather"}}
chat_response = chat_completion_request(
    messages, tools=tools
)
# 解析返回的JSON数据，获取助手的回复消息
assistant_message = chat_response.json()["choices"][0]["message"]

# 将助手的回复消息添加到messages列表中
messages.append(assistant_message)

# 判断并执行tool
if assistant_message.get('tool_calls'):
    results = excute_tool(assistant_message)
    messages.append({"role": "tool", "tool_call_id":assistant_message['tool_calls'][0]['id'], 
                     "name":assistant_message['tool_calls'][0]['function']['name'],  "content": json.dumps(results,ensure_ascii=False)})
    chat_response = chat_completion_request(
    messages, tools=tools)
    # 解析返回的JSON数据，获取助手的回复消息
    assistant_message = chat_response.json()["choices"][0]["message"]
    
    # 将助手的回复消息添加到messages列表中
    messages.append(assistant_message)

pretty_print_conversation(messages)

system: 你是一个天气预报大师，你需要先检查用户的描述中是否有提供所在地

user: 今天天气怎么样?

assistant[content]: 对不起，您需要告诉我您所在的地方才能查询当天的天气情况。



In [9]:
messages.append({'role': 'user', 'content': '广州市吧'})

# 使用定义的chat_completion_request函数发起一个请求，传入messages和functions作为参数
# tool_chioce = {"type": "function", "function": {"name": "get_next_x_days_weather"}}
chat_response = chat_completion_request(
    messages, tools=tools
)
# 解析返回的JSON数据，获取助手的回复消息
assistant_message = chat_response.json()["choices"][0]["message"]

# 将助手的回复消息添加到messages列表中
messages.append(assistant_message)

# 判断并执行tool
if assistant_message.get('tool_calls'):
    results = excute_tool(assistant_message)
    messages.append({"role": "tool", "tool_call_id":assistant_message['tool_calls'][0]['id'], 
                     "name":assistant_message['tool_calls'][0]['function']['name'],  "content": json.dumps(results,ensure_ascii=False)})
    chat_response = chat_completion_request(
    messages, tools=tools)
    # 解析返回的JSON数据，获取助手的回复消息
    assistant_message = chat_response.json()["choices"][0]["message"]
    
    # 将助手的回复消息添加到messages列表中
    messages.append(assistant_message)

pretty_print_conversation(messages)

system: 你是一个天气预报大师，你需要先检查用户的描述中是否有提供所在地

user: 今天天气怎么样?

assistant[content]: 对不起，您需要告诉我您所在的地方才能查询当天的天气情况。

user: 广州市吧

assistant[tool_calls]: [{'function': {'arguments': '{"location":"广州"}', 'name': 'get_current_weather'}, 'id': 'call_202408270825293a1a1d1b2992448c', 'index': 0, 'type': 'function'}]

function (get_current_weather): {"date": "2024-08-27", "wea": "多云转阵雨", "wea_img": "lei", "wea_day": "多云", "wea_day_img": "yun", "wea_night": "阵雨", "wea_night_img": "lei", "tem_day": "35", "tem_night": "26", "win": "微风<3级"}

assistant[content]: 广州市今天的天气情况是多云转阵雨，白天多云，最高温度为35度，夜晚有阵雨，最低温度为26度。



In [10]:
messages.append({'role': 'user', 'content': '如果想知道未来3天广州市的天气呢'})

# 使用定义的chat_completion_request函数发起一个请求，传入messages和functions作为参数
# tool_chioce = {"type": "function", "function": {"name": "get_next_x_days_weather"}}
chat_response = chat_completion_request(
    messages, tools=tools
)
# 解析返回的JSON数据，获取助手的回复消息
assistant_message = chat_response.json()["choices"][0]["message"]

# 将助手的回复消息添加到messages列表中
messages.append(assistant_message)

# 判断并执行tool
if assistant_message.get('tool_calls'):
    results = excute_tool(assistant_message)
    messages.append({"role": "tool", "tool_call_id":assistant_message['tool_calls'][0]['id'], 
                     "name":assistant_message['tool_calls'][0]['function']['name'],  "content": json.dumps(results,ensure_ascii=False)})
    chat_response = chat_completion_request(
    messages, tools=tools)
    # 解析返回的JSON数据，获取助手的回复消息
    assistant_message = chat_response.json()["choices"][0]["message"]
    
    # 将助手的回复消息添加到messages列表中
    messages.append(assistant_message)

pretty_print_conversation(messages)

system: 你是一个天气预报大师，你需要先检查用户的描述中是否有提供所在地

user: 今天天气怎么样?

assistant[content]: 对不起，您需要告诉我您所在的地方才能查询当天的天气情况。

user: 广州市吧

assistant[tool_calls]: [{'function': {'arguments': '{"location":"广州"}', 'name': 'get_current_weather'}, 'id': 'call_202408270825293a1a1d1b2992448c', 'index': 0, 'type': 'function'}]

function (get_current_weather): {"date": "2024-08-27", "wea": "多云转阵雨", "wea_img": "lei", "wea_day": "多云", "wea_day_img": "yun", "wea_night": "阵雨", "wea_night_img": "lei", "tem_day": "35", "tem_night": "26", "win": "微风<3级"}

assistant[content]: 广州市今天的天气情况是多云转阵雨，白天多云，最高温度为35度，夜晚有阵雨，最低温度为26度。

user: 如果想知道未来3天广州市的天气呢

assistant[tool_calls]: [{'function': {'arguments': '{"location":"广州","x":3}', 'name': 'get_next_x_days_weather'}, 'id': 'call_20240827082548e8f57718fa054e0f', 'index': 0, 'type': 'function'}]

function (get_next_x_days_weather): [{"date": "2024-08-28", "wea": "阵雨转中到大雨", "wea_img": "lei", "wea_day": "阵雨", "wea_day_img": "lei", "wea_night": "中到大雨", "wea_night_img": "yu", "tem_day":